# Optimization and root finding (scipy.optimize)

```
minimize(fun, x0[, args, method, jac, hess, ...]) 	Minimization of scalar function of one or more variables.
minimize_scalar(fun[, bracket, bounds, ...]) 	Minimization of scalar function of one variable.
OptimizeResult 	Represents the optimization result.
```
The minimize function supports the following methods:
```
    minimize(method=’Nelder-Mead’)
    minimize(method=’Powell’)
    minimize(method=’CG’)
    minimize(method=’BFGS’)
    minimize(method=’Newton-CG’)
    minimize(method=’L-BFGS-B’)
    minimize(method=’TNC’)
    minimize(method=’COBYLA’)
    minimize(method=’SLSQP’)
    minimize(method=’dogleg’)
    minimize(method=’trust-ncg’)
```
The minimize_scalar function supports the following methods:
```
    minimize_scalar(method=’brent’)
    minimize_scalar(method=’bounded’)
    minimize_scalar(method=’golden’)
```
More information at http://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

## Optimization of multiple variables

In [2]:
def f_simple(x):
    return (x[0] - 10.0)**2 + (x[1] + 5.0)**2+x[0]**2

In [21]:
from scipy.optimize import minimize
res = minimize(f_simple,[0,0],method='BFGS', 
         options={'disp': True})
print res.x

Optimization terminated successfully.
         Current function value: 50.000000
         Iterations: 2
         Function evaluations: 20
         Gradient evaluations: 5
[ 5.00000003 -4.99999999]


In [12]:
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [22]:
import numpy as np
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='CG', #Conjugate gradient method
               options={'disp': True})
print res.x

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 89
         Function evaluations: 1162
         Gradient evaluations: 166
[ 1.00000012  1.00000024  1.00000049  1.00000098  1.00000197]


## Line search

In [23]:
def f(x):
    return 2+(1-x)**2

In [26]:
from scipy.optimize import minimize_scalar
minimize_scalar(f,method='golden')

  fun: 2.0
    x: 0.999999992928981
 nfev: 44

In [27]:
minimize_scalar(f,method='brent')

  fun: 2.0
 nfev: 5
  nit: 4
    x: 0.99999998519